### Experiments with DBSCAN algorithm for Gregorian chant repertoire tradition detection
* distance matrix construction (parameter metric='precomputed')
* parameters:
    -  eps
    -  min_samples
    
* check for noise samples

In [14]:
# Imports
import numpy as np
import pandas as pd

from sklearn.cluster import DBSCAN

import matplotlib.pyplot as plt
from itertools import combinations
from random import sample, seed
from collections import Counter, OrderedDict
import random

import pickle
import lzma

In [3]:
# Read data
responsories_all = pd.read_csv('../data/all-ci-responsories.csv', usecols=['cantus_id', 'incipit', 'siglum', 'source_id', 'feast_id'], dtype={'cantus_id':"str"})
antiphons_all = pd.read_csv('../data/all-ci-antiphons.csv', usecols=['cantus_id', 'incipit', 'siglum', 'source_id', 'feast_id'], dtype={'cantus_id':"str"})

sources = pd.read_csv('../data/sources-with-provenance-ids-and-two-centuries.csv', usecols=['provenance_id', 'drupal_path', 'siglum', 'cursus', 'num_century'])
feasts = pd.read_csv('../data/feast.csv', usecols=['id', 'name'])

chants = pd.concat([responsories_all, antiphons_all])

In [4]:
# Sources compare metric
def jaccard_dist(a : list, b : list):
    '''
    Function returns value of Jaccard metrics applied on two sets
    '''
    if len(set(a).union(set(b))) != 0:
        return (len(set(a).intersection(set(b))) / len(set(a).union(set(b))))
    else:
        return 0

In [6]:
# Source translate to int for smooth matrix indexing 
source_dict = OrderedDict()
i = 0
for id in sources['drupal_path']:
    source_dict[id] = i
    i += 1

In [9]:
def get_distance_matrix_all(compare_func, sources):
    ''' 
    Constructs distane matrix for given sources (indexing in given order)
    from all data (all feasts)
    '''
    source_chants_dict = {}

    for source_id in sources:
        filt_source = chants['source_id'] == source_id
        source_chants_dict[source_id] = (chants[filt_source]['cantus_id']).tolist()
    
    distance_matrix = np.zeros([len(sources), len(sources)])
    for s_i in sources:
        for s_j in sources:
            distance_matrix[source_dict[s_i], source_dict[s_j]] = 1 - compare_func(source_chants_dict[s_i], source_chants_dict[s_j])
    
    return distance_matrix

In [10]:
def shuffle(perm, sources):
    '''
    Shuffles sources based on given permutation of indexess
    '''
    shuffled_sources = [sources[j] for j in perm]
    return shuffled_sources

def unshuffle(perm, labels):
    '''
    Restore original order of elemenets from permutation,
    that was used for their shuffling
    '''
    unshuffled_labels = np.zeros(len(labels))
    for i, j in enumerate(perm):
        unshuffled_labels[j] = labels[i]
    return unshuffled_labels

### Experiments

First focus on amount o samples classified as noise (label == -1)

In [11]:
# Used options for parameters search and random states for shuffling sources
EPS_OPTIONS = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
MIN_SAMPLES_OPTIONS = [2, 3, 4, 5, 6, 7]
random_states = [i for i in range(1, 151)]

In [13]:
# Run on all data graph = look for noise

dict_all_labels = {}
dict_all_noise_counts = {}

rand_idx = 0
for eps in EPS_OPTIONS:
    for min_sample in MIN_SAMPLES_OPTIONS:
        perm = list(range(len(source_dict)))
        random.seed(random_states[i])
        random.shuffle(perm)
        shuff_sources = shuffle(perm, list(source_dict.keys()))
        distance_matrix = get_distance_matrix_all(jaccard_dist, shuff_sources)
        clustering = DBSCAN(eps=eps, min_samples=min_sample)
        clustering.fit(distance_matrix)

        labels = unshuffle(perm, clustering.labels_)
        
        unique, counts = np.unique(labels, return_counts=True)
        counts_dict = dict(zip(unique, counts))
        dict_all_noise_counts[(eps, min_sample)] = counts_dict[-1]
        if counts_dict[-1] < 125:
            print('Setup that leads to less than half of all samples noisy:', eps, min_sample, counts_dict[-1])
            
        dict_all_labels[(eps, min_sample)] = labels
    rand_idx += 1

Setup that leads to less than half of all samples noisy: 0.9 2 112


In [15]:
# Saved computed dict_all_labels and dict_all_noise_counts

#with lzma.open("saved_results/dbscan/dict_all_labels.txt", "rb") as model_file:
#    dict_all_labels = pickle.load(model_file)
#with lzma.open("saved_results/dbscan/dict_all_noise_counts.txt", "rb") as model_file:
#    dict_all_noise_counts = pickle.load(model_file)

Let's look at numbers of noise while running computations on each of feasts

In [22]:
# Select "more serious" feasts - having more than 10 records of antiphons or responsories
freq_of_feasts = chants['feast_id'].value_counts()
feasts_without_little = freq_of_feasts.drop(freq_of_feasts[freq_of_feasts.values < 10].index).index.tolist()
print(feasts_without_little)

['feast_1202', 'feast_1416', 'feast_0552', 'feast_0093', 'feast_0198', 'feast_0227', 'feast_1283', 'feast_0719', 'feast_0476', 'feast_0983', 'feast_1548', 'feast_0091', 'feast_0500', 'feast_0343', 'feast_1204', 'feast_1150', 'feast_0994', 'feast_0258', 'feast_0067', 'feast_0752', 'feast_1200', 'feast_1125', 'feast_0475', 'feast_1056', 'feast_0055', 'feast_0031', 'feast_0033', 'feast_0472', 'feast_0473', 'feast_1497', 'feast_0933', 'feast_0438', 'feast_0291', 'feast_0390', 'feast_1262', 'feast_0162', 'feast_1115', 'feast_0234', 'feast_1321', 'feast_0332', 'feast_0445', 'feast_0467', 'feast_0416', 'feast_0581', 'feast_0470', 'feast_1343', 'feast_1211', 'feast_0959', 'feast_0360', 'feast_0132', 'feast_0577', 'feast_0625', 'feast_0437', 'feast_0444', 'feast_1765', 'feast_0245', 'feast_0287', 'feast_1249', 'feast_0415', 'feast_0474', 'feast_0658', 'feast_0339', 'feast_0725', 'feast_0696', 'feast_0761', 'feast_0323', 'feast_1501', 'feast_1266', 'feast_0325', 'feast_0330', 'feast_1352', 'feas

In [18]:
def get_distance_matrix_one_feast(compare_func, sources, sources_dict, source_chants_dict):
    ''' 
    Constructs distance matrix for sources with specified subset of chants
    (for example by feast)
    '''
    distance_matrix = np.zeros([len(sources), len(sources)])
    for s_i in sources:
        for s_j in sources:
            distance_matrix[sources_dict[s_i], sources_dict[s_j]] = 1 - compare_func(source_chants_dict[s_i], source_chants_dict[s_j])
    
    return distance_matrix

In [25]:
# Look at each of bigger feasts and check noise samples
# Prints out those with less than half samples classified as noisy
print("Feast\t\tin_sources\tnoise_samples")
at_leas_half = 0
skiiped = 0
for feast_id in feasts_without_little:
    used_sources = []
    source_chants_dict = {}
    filt_feast = chants['feast_id'] == feast_id
    chants_of_feast = chants[filt_feast]
    for source_id in sources['drupal_path']:
        filt_source = chants_of_feast['source_id'] == source_id
        if (chants_of_feast[filt_source]['cantus_id']).tolist() != []:
            used_sources.append(source_id)
            source_chants_dict[source_id] = (chants_of_feast[filt_source]['cantus_id']).tolist()

    if used_sources == []:
        skiiped += 1
        continue
    source_dict = OrderedDict()
    i = 0
    for id in used_sources:
        source_dict[id] = i
        i += 1
    
    distance_matrix = get_distance_matrix_one_feast(jaccard_dist, used_sources, source_dict, source_chants_dict)
    clustering = DBSCAN(eps=0.2, min_samples=9)
    clustering.fit(distance_matrix)
    
    unique, counts = np.unique(clustering.labels_, return_counts=True)
    counts_dict = dict(zip(unique, counts))

    if(counts_dict[-1] < (len(used_sources)/2)):
        at_leas_half += 1
        print(str(feasts[feasts['id'] == feast_id]['name'].values[0])+'\t'+str(len(used_sources))+'\t'+str(counts_dict[-1]))

Feast		in_sources	noise_samples
Antiphonae Majores	104	37
Briccii	81	36
Fer. 2 Hebd. 3 Adv.	98	43
Fer. 3 Hebd. 3 Adv.	99	48
Fer. 3 de Passione	106	31
Fer. 4 Hebd. 4 Quad.	105	35
Dom. 3 p. Pent.	100	42
Fer. 2 Hebd. 3 Quad.	106	33
Dom. 4 p. Pent.	100	48
Fer. 2 Hebd. 4 Quad.	108	32
Fer. 5 post Cineres	105	48
Fer. 4 Hebd. 1 Quad.	101	38
Fer. 6 Hebd. 1 Quad.	102	35
Fer. 4 Hebd. 2 Quad.	107	24
Fer. 4 de Passione	107	44
Dom. 10 p. Pent.	102	39
Dom. 22 p. Pent.	99	33
Fer. 6 Hebd. 4 Quad.	105	29
Dom. 11 p. Pent.	103	47
Fer. 2 Hebd. 2 Quad.	110	29
Dom. 21 p. Pent.	99	49
Fer. 3 Hebd. 2 Quad.	111	33
Fer. 6 Hebd. 2 Quad.	108	27
Fer. 3 Hebd. 1 Quad.	105	51
Dom. 18 p. Pent.	98	35
Fer. 6 de Passione	108	33
Fer. 6 Hebd. 1 Adv.	97	37
Fer. 3 Hebd. 3 Quad.	106	27
Fer. 5 de Passione	107	28
Fer. 4 Hebd. 1 Adv.	98	35
Fer. 4 Hebd. 3 Quad.	107	22
Fer. 5 Hebd. 1 Adv.	98	39
Dom. 23 p. Pent.	98	31
Fer. 2 Hebd. 2 Adv.	95	26
Dom. 24 p. Pent.	93	38
Fer. 4 Hebd. 2 Adv.	98	25
Fer. 6 Hebd. 2 Adv.	97	21
Fer. 3 Hebd. 2 A

In [30]:
print('number of feasts with at least 10 chants:', len(feasts_without_little))
print('number of feasts from those with at least 10 chants that has less than half samples noise:', at_leas_half)

number of feasts with at least 10 chants: 918
number of feasts from those with at least 10 chants that has less than half samples noise: 48
